In [25]:
import pandas as pd
from pandas.testing import assert_frame_equal
from typing import Tuple

from pipelines.utils.config import get_env_var_as_path

In [11]:
index_path = get_env_var_as_path("PATH_DATA_FINAL")
input_path = get_env_var_as_path("PATH_DATA_INTERIM_RAW2PARQUET")
index_path, input_path

(PosixPath('/home/paulo/Documents/mba_tcc/data/2_final'),
 PosixPath('/home/paulo/Documents/mba_tcc/data/1_interim/0__raw_to_parquet'))

In [12]:
index_df = pd.read_parquet(index_path / "files_index.parquet")

In [13]:
raw_data_file = index_df.iloc[0].to_dict()
raw_data_file

{'file_path': '/home/paulo/Documents/mba_tcc/data/0_raw/UCR_Anomaly_FullData/001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_52620.txt',
 'file_name': '001_UCR_Anomaly_DISTORTED1sddb40_35000_52000_52620.txt',
 'file_number': 1,
 'mnemonic': 'DISTORTED1sddb40',
 'training_index_end': 35000,
 'anomaly_index_start': 52000,
 'anomaly_index_end': 52620}

In [16]:
data_file_path = (input_path / raw_data_file["file_name"]).with_suffix(".parquet")
data_file = pd.read_parquet(data_file_path)

In [19]:
train_df = data_file.loc[:raw_data_file["training_index_end"] - 1]
test_df = data_file.loc[raw_data_file["training_index_end"]:]

In [23]:
def df_slice(df, index_start=0, index_end=None) -> Tuple[pd.DataFrame, pd.DataFrame]:
    last_df_index = len(df)
    if not index_end:
        index_end = last_df_index
        
    return df.loc[index_start:index_end - 1]

In [27]:
train_df_func = df_slice(data_file, index_end=raw_data_file["training_index_end"]-1)
test_df_func = df_slice(data_file, index_start=raw_data_file["training_index_end"])

In [28]:
assert_frame_equal(train_df, train_df_func)

In [29]:
assert_frame_equal(test_df, test_df_func)

In [31]:
assert len(data_file) == len(train_df) + len(test_df)

In [32]:
assert len(data_file) == len(train_df_func) + len(test_df_func)